In [3]:
import boo
print(boo.__version__)


1.0.0


In [17]:
import numpy as np
import MDAnalysis as mda
from scipy.spatial import Delaunay
import boo  # note: the module is imported as "boo"
import matplotlib.pyplot as plt

u = mda.Universe("mac40x40.data", "mac.dump", format="LAMMPSDUMP", atom_style='id type x y z')

# Select a frame – here we use the first frame
u.trajectory[0]
pos2d = u.atoms.positions[:, :2]  # shape (N, 2)
pos = np.hstack((pos2d, np.zeros((pos2d.shape[0], 1))))

tri = Delaunay(pos2d)
bonds_set = set()
for simplex in tri.simplices:
    # For each triangle, add all unique bonds (each bond only once)
    for i in range(3):
        for j in range(i+1, 3):
            # Sort indices so that (i, j) and (j, i) are not both added
            bond = tuple(sorted((simplex[i], simplex[j])))
            bonds_set.add(bond)
# Convert to a (M, 2) array
bonds = np.array(list(bonds_set), dtype=int)

print("Number of bonds:", len(bonds))

box = np.array([40.0, 40.0, 0.0])

qlms = boo.bonds2qlm(pos, bonds, l=6, periods=box)

maxdist = 20.0  # maximum distance to consider
Nbins = 50      # number of bins
is_center = np.ones(len(pos), dtype=bool)
num, den, r_bins = boo.gG_l(pos, qlms, Nbins=50,is_center=is_center, maxdist=maxdist)

g6 = np.where(den > 0, num / den, 0)

plt.figure(figsize=(12, 6))
plt.plot(r_bins, g6, 'o-', label=r'$g_6(r)$')
plt.xlabel("Distance r (Å)")
plt.ylabel(r"$g_6(r)$")
plt.title("Orientational Correlation Function (Hexatic Order)")
plt.legend()
plt.show()

Number of bonds: 1811


c:\Users\David\AppData\Local\Programs\Python\Python312\Lib\site-packages\MDAnalysis\coordinates\LAMMPS.py:751: UserWarning: Reader has no dt information, set to 1.0 ps
  ts.data["time"] = step_num * ts.dt
c:\Users\David\AppData\Local\Programs\Python\Python312\Lib\site-packages\numba\np\ufunc\dufunc.py:288: RuntimeWarning: invalid value encountered in periodify
  return super().__call__(*args, **kws)


AssertionError: 